# Starbucks Object Detection: Multiple Image Extraction

<img src="https://pages.databricks.com/rs/094-YMS-629/images/SBUX-DB.png"/>

This notebook provides a simplified example of object detection (e.g. coffee, cups, chair, etc.) within images and videos.  In addition to use *Databricks Runtime for ML - GPU*, we are also working with the [Image AI](https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl) library.

## Setup
There are additional installations to include the **pre-trained** models as part of **[ImageAI](https://github.com/OlafenwaMoses/ImageAI/blob/master/imageai/Detection/VIDEO.md):**
* **Yolo V3**: You only look once (YOLO) is a state-of-the-art, real-time object detection system; we are using V3 as noted in [YOLOv3: An Incremental Improvement](https://pjreddie.com/media/files/papers/YOLOv3.pdf) (using this model for images)
* **Resnet**: Residual Network model (in h5 format) that we are using for feature extraction (using this model for videos)

In [0]:
# Configure file paths
source_images_path = "/mnt/tardis6/images/starbucks/samples/"
processing_images_path = "/tmp/dennylee/cafe/images/starbucks/samples/"
execution_path = "/dbfs/tmp/dennylee/cafe/"  # Matches following cell's shell script to copy the pre-trained models
images_path = "/dbfs" + processing_images_path
model_name = "yolo.h5"
verbose = True

In [0]:
%sh
## Install the ImageAI pre-trained models: YoloV3, ResNet
# wget -P /dbfs/tmp/dennylee/cafe/ https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5
# wget -P /dbfs/tmp/dennylee/cafe/ https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5

In [0]:
# Configure TensorFlow for Auto-Growth
import tensorflow as tf
tf.set_random_seed(42) # For reproducibility

config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.5
tf.Session(config=config)

# Configure MLflow Experiment
mlflow_experiment_id = 2102416

In [0]:
# Include HorovodRunner from Spark DL Pipelines
from sparkdl import HorovodRunner

# Include Horovod Keras 
import horovod.keras as hvd

import keras
from keras import backend as K
from keras.callbacks import TensorBoard

In [0]:
## ------------------------------
## Miscellaneous Functions
## ------------------------------
# displayVid(): Shows video from mounted cloud storage
def displayVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/files/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayDbfsVid(): Shows video from DBFS
def displayDbfsVid(filepath):
  return displayHTML("""
  <video width="480" height="320" controls>
  <source src="/dbfs/%s" type="video/mp4">
  </video>
  """ % filepath)

# displayImg(): Shows image from dbfs/cloud storage
def displayImg(filepath):
  dbutils.fs.cp(filepath, "FileStore/%s" % filepath)
  return displayHTML("""
  <img src="/files/%s" width="800">
  """ % filepath)

## Extracting Coffee Objects from Multiple Images

Let's apply the same model but this time run it against multiple images form our `samples`

In [0]:
# Copy the files to `tmp` folder to ensure we're working with new files
dbutils.fs.rm(processing_images_path, recurse=True)
dbutils.fs.cp(source_images_path, processing_images_path, recurse=True)

Out[15]: True

In [0]:
from pyspark.ml.image import ImageSchema

images_samples = ImageSchema.readImages(processing_images_path)
display(images_samples)

image
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/1.jpeg, 2808, 4200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/10.jpg, 1024, 634, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/11.jpg, 700, 467, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/13.jpg, 430, 760, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/2.jpg, 563, 750, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/21.jpg, 520, 780, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/3.jpg, 3333, 5000, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/4.jpg, 886, 898, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/54.jpg, 720, 1280, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"
"List(dbfs:/tmp/dennylee/cafe/images/starbucks/samples/6.jpg, 865, 1240, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)"


In [0]:
# 
# Detect and Extract Images using Yolo (local mode)
#
def detectExtractImagesYolo_local():
  
  # Import ObjectDetection from ImageAI
  from imageai.Detection import ObjectDetection
  import os
  import h5py
  import shutil
  import hashlib
  import imghdr
  import time
  import mlflow
  

  with mlflow.start_run(experiment_id = mlflow_experiment_id) as run:
    # Initial Start time
    initial_start_time = time.time()
    
    # Log MLflow Parameters
    mlflow.log_param("processing", "local [allow] (y)")
    
    # Loop through images within the folder
    for image_name in os.listdir(images_path):

      # Only include images (not the output)
      if "-objects" not in image_name:
        # Record Start time
        start_time = time.time()

        print("********************************")
        print("Processing: %s " % image_name)
        print("********************************")

        # Use ObjectDetection using the YoloV3 model
        detector = ObjectDetection()
        detector.setModelTypeAsYOLOv3()
        detector.setModelPath(os.path.join(execution_path , model_name))
        detector.loadModel()

        # Remove directory if exists
        image_output_fullpath = images_path + image_name + "-objects"
        if os.path.exists(image_output_fullpath) and os.path.isdir(image_output_fullpath):
          shutil.rmtree(image_output_fullpath)

        # Detect Objects in the image
        detections, objects_path = detector.detectObjectsFromImage(
          input_image=os.path.join(images_path, image_name), 
          output_image_path=os.path.join(images_path, image_name), 
          minimum_percentage_probability=30,  
          extract_detected_objects=True
        )  

        # Print out the images extracted, box points, and percentage probability
        if verbose:
          for eachObject, eachObjectPath in zip(detections, objects_path):
              print(eachObject["name"] , " : " , eachObject["percentage_probability"], " : ", eachObject["box_points"] )
              print("Object's image saved in " + eachObjectPath)
              print("--------------------------------")                  
              
        # Record Complete Time and determine Elapsed Time
        complete_time = time.time()
        elapsed_time = complete_time - start_time      
        
        # Log MLflow Metrics
        mlflow.log_metric("duration", elapsed_time)
        
    # Overall completed time
    final_complete_time = time.time()
    overall_elapsed_time = final_complete_time - initial_start_time
    
    # Log Total Duration
    mlflow.log_metric("total duration", overall_elapsed_time)
    
  # Complete MLflow Run
  mlflow.end_run()  

In [0]:
# Clears out temp folder and resets images to original (uncomment if doing multiple runs from this cell)
#dbutils.fs.rm(processing_images_path, recurse=True)
#dbutils.fs.cp(source_images_path, processing_images_path, recurse=True)

# Detect and Extract Images Locally
detectExtractImagesYolo_local()

********************************
Processing: 1.jpeg 
********************************
dining table : 80.6664526462555 : (287, 1581, 4050, 2730)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/1.jpeg-objects/dining table-1.jpg
--------------------------------
cup : 99.97636675834656 : (1200, 291, 2720, 2717)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/1.jpeg-objects/cup-2.jpg
--------------------------------
person : 97.73234128952026 : (2864, 528, 3647, 1837)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/1.jpeg-objects/person-3.jpg
--------------------------------
person : 99.64783787727356 : (53, 0, 1399, 2740)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/1.jpeg-objects/person-4.jpg
--------------------------------
********************************
Processing: 10.jpg 
********************************
cake : 75.16087293624878 : (149, 217, 504, 859)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/10.jpg-objects/cake-1.jpg
--------------------------------
cup : 85.78367829322815 : (154, 237, 509, 900)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/10.jpg-objects/cup-2.jpg
--------------------------------
bottle : 32.336825132369995 : (468, 201, 617, 720)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/10.jpg-objects/bottle-3.jpg
--------------------------------
bottle : 99.3109405040741 : (0, 492, 212, 897)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/10.jpg-objects/bottle-4.jpg
--------------------------------
********************************
Processing: 11.jpg 
********************************
cup : 99.84104633331299 : (118, 88, 387, 612)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/11.jpg-objects/cup-1.jpg
--------------------------------
********************************
Processing: 13.jpg 
********************************
vase : 64.81115221977234 : (151, 106, 385, 420)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/13.jpg-objects/vase-1.jpg
--------------------------------
vase : 74.03401136398315 : (372, 10, 588, 403)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/13.jpg-objects/vase-2.jpg
--------------------------------
dining table : 90.12545347213745 : (0, 285, 760, 424)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/13.jpg-objects/dining table-3.jpg
--------------------------------
potted plant : 86.57552003860474 : (126, 6, 398, 421)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/13.jpg-objects/potted plant-4.jpg
--------------------------------
cup : 57.437920570373535 : (151, 106, 385, 420)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/13.jpg-objects/cup-5.jpg
--------------------------------
********************************
Processing: 2.jpg 
********************************
vase : 32.435059547424316 : (254, 104, 501, 541)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/2.jpg-objects/vase-1.jpg
--------------------------------
cup : 69.00558471679688 : (254, 104, 501, 541)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/2.jpg-objects/cup-2.jpg
--------------------------------
bottle : 77.2420585155487 : (473, 52, 592, 442)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/2.jpg-objects/bottle-3.jpg
--------------------------------
bottle : 99.52913522720337 : (109, 259, 314, 557)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/2.jpg-objects/bottle-4.jpg
--------------------------------
********************************
Processing: 21.jpg 
********************************
cup : 61.90655827522278 : (306, 126, 431, 365)
Object's image saved in /dbfs/tmp/dennylee/cafe/images/starbucks/samples/21.jpg-objects/cup-1.jpg
--------------------------------
cup : 97.8

In [0]:
# Close TensorFlow session
tf.Session.close

Out[4]: <function tensorflow.python.client.session.BaseSession.close(self)>

## Observations:

TensorFlow by default uses up nearly all of available GPU memory visible to the *process* not the lifetime of the session object.  That is, [if you want memory to be freed, you'll have to exit the Python interpretor, not just close the session](https://github.com/tensorflow/tensorflow/issues/17048#issuecomment-367948448).  This is done to more efficiently use the relatively precious GPU memory resources on the devices by reducing memory fragmentation.

A potential way to minimize the impact of this issue is by setting "[Allowing GPU memory growth](https://www.tensorflow.org/guide/using_gpu#allowing_gpu_memory_growth)" as noted in cell 5 of this notebook.  As noted in the following graph, the duration of each image processing (object detection) takes subsequently longer due to less and less GPU memory resources being available.

![](https://pages.databricks.com/rs/094-YMS-629/images/image-processing-duration-via-allow-growth-tf.png)

The top run is running *without* `allow_growth` thus taking even longer to complete with the **total duration** of 6530.943245649338s.  In fact, multiple runs *without* `allow_growth` resulted in out of memory errors.  The bottom run is running *with* `allow_growth`; while each subsequent run takes longer, it isn't as extreme as the top run with a **total duration** of 1523.47892117s. If you have access to the **Databricks Demo Cluster**, you can see the full set of statistics within the MLflow HorovodRunner-DL [comparitive runs](https://demo.cloud.databricks.com/#mlflow/metric/duration?runs=[%22e0772defc96147c4a7c78dca2fc5f561%22,%229ab9b378a7c941878f6c29b498e23541%22]&experiment=2102416).

<br/>

![](https://pages.databricks.com/rs/094-YMS-629/images/Three-repeated-ImageAI-runs.png)

This can also be seen when executing repeated runs as noted in the preceding graphs.

On the left is when this notebook (cell 5) does *not* have `allow_growth`:  
* While the first one completes without any issues
* The second run appears to be paging GPU memory and thus running significantly slower.  Each image is taking increasingly longer to complete and the total duration to process all images is now 6178.2s instead of the original 1522.1s. 
* The third run is not on the chart because the GPU has run out of memory and failed.

On the right is when this notebook (cell 5) *does* have `allow_growth`:
* All three runs complete approximately the same time with a hiccup for the third run for a single image.
* All three runs last image takes approx 151s to complete processing
* All three runs total duration to process all images in approx 1570s

## Roadmap
We are starting with ImageAI due to the existing pre-trained models. Future plans include:
* Using `keras-yolov3` directly so we can train our own models to be more Starbucks specific (coffee, baked goods, cups, etc.) 
* Scale up to improve the speed of these models
* Scale out to handle more images and videos in a *distributed* fashion